# PROYECTO INTEGRADOR: STEAM
## Funciones para los endpoints

El tratamiento de los datasets previo a las funciones, se realiza en el archivo 04_tratamiento_para_funciones.ipynb , por favor revisar ese archivo para comprender las modificaciones realizadas, allí podrá encontrar el tratamiento de cada dataset para intentar disminuir la cantidad de datos para optimizar la API.

In [24]:
#Importamos las librerías 
import pandas as pd

In [25]:
#Dataset para función 1
df_generos_horas_jugadas=pd.read_csv('Data/generos_horas_jugadas.csv')

In [26]:
#Dataset para función 2
df_usuario_horas_jugadas=pd.read_parquet('Data/usuario_horas_jugadas.parquet')

In [27]:
#Dataset para función 3
df_recomendaciones_positivas=pd.read_csv('Data/recomendaciones_positivas.csv')

In [28]:
#Dataset para función 4
df_recomendaciones_negativas=pd.read_csv('Data/recomendaciones_negativas.csv')

In [29]:
#Dataset para función 5
df_sentiment_analysis=pd.read_csv('Data/sentiment_analysis.csv')

## **1) Función PlayGenRe**

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
#Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

En el tratamiento se realizó un merge entre 2 de los datasets, luego un  group_by por genres y year_release, por lo que en esta instancia se debe seleccionar el género y el máximo año en función de los playtime sumados.Además, se eliminaron las columnas innecesarias.

In [30]:
def PlayTimeGenre(genero):
    '''
    Debe devolver año con mas horas jugadas para dicho género.
    '''
    #Vamos a utilizar el dataframe df_generos_horas_jugadas, que recordar que ya se encuentra agrupado en el archivo 04_tratamiento_para_funciones.ipynb
    
    genero=genero.lower()

    # Filtramos el DataFrame por el género especificado
    df_filtrado = df_generos_horas_jugadas[df_generos_horas_jugadas['genres'].str.lower() == genero]

    # Encontrar el año con la máxima suma de horas jugadas
    año_max_horas = df_filtrado.loc[df_filtrado['playtime_forever'].idxmax()]

    return("El año con más horas jugadas para el genero " + str(genero).capitalize() + " es " + str(año_max_horas['release_year']))


In [31]:
#Verificamos el correcto funcionamiento de la función
PlayTimeGenre('Action')

'El año con más horas jugadas para el genero Action es 2012'

In [32]:
#Verificamos el correcto funcionamiento de la función
PlayTimeGenre('RPG')

'El año con más horas jugadas para el genero Rpg es 2011'

## **2) Función UserFonGenre**

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
#Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Como en el caso anterior, se hizo un merge análogo, luego un group by en función de genres, release_year y el user_id para sumar los playtime para encontrar el usuario con más horas jugadas, y luego poder mostrar la lista por año para ese usuario. Además, se eliminaron las columnas innecesarias. Recordar que los group by se realizaron en el archivo 04_tratamiento_para_funciones.ipynb.  

In [33]:
def UserForGenre(genero: str):
    '''
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    
    '''

    genero=genero.lower()
    # Filtra el dataframe df_final2 por el género solicitado
    df_filtrado = df_usuario_horas_jugadas.loc[df_usuario_horas_jugadas["genres"].str.lower() == genero]
    # Comprueba si hay datos para el género solicitado

    
    if df_filtrado.empty:
        return "El genero ingresado no es válido"
    
    # Agrupa el dataframe filtrado por el usuario y suma las horas jugadas
    df_user = df_filtrado.groupby("user_id")["playtime_forever"].sum()

    # Obtiene el usuario con más horas jugadas para el género solicitado
    user = df_user.idxmax()

    # Filtra el dataframe filtrado por el usuario obtenido
    df_user_genre = df_filtrado.loc[df_filtrado["user_id"] == user]

    # Agrupa el dataframe filtrado por el año y suma las horas jugadas
    df_user_year = df_user_genre.groupby("release_year")["playtime_forever"].sum()

    # Crea una lista con la acumulación de horas jugadas por año
    hours = [{"Año": year, "Horas": hours} for year, hours in df_user_year.items()]
    
    # Devuelve el usuario con más horas jugadas para el género solicitado y la lista de la acumulación de horas jugadas por año en formato JSON
    return {f"Usuario con más horas jugadas para el género {genero}": user, "Horas jugadas": hours}

In [34]:
UserForGenre('Action')

{'Usuario con más horas jugadas para el género action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 217},
  {'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 2001, 'Horas': 13},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2003, 'Horas': 7673},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2006, 'Horas': 896},
  {'Año': 2007, 'Horas': 112784},
  {'Año': 2008, 'Horas': 224},
  {'Año': 2009, 'Horas': 108326},
  {'Año': 2010, 'Horas': 78083},
  {'Año': 2011, 'Horas': 154896},
  {'Año': 2012, 'Horas': 379522},
  {'Año': 2013, 'Horas': 120461},
  {'Año': 2014, 'Horas': 130691},
  {'Año': 2015, 'Horas': 307511},
  {'Año': 2016, 'Horas': 29576},
  {'Año': 2017, 'Horas': 43327}]}

## **3) Función UsersRecommend**

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

En este caso, para obtener el dataset se hizo un merge, se filtraron los recommend=True y los valores de la columna sentimiento igual a 1 y 2, por lo que ese filtro no se encuentra incluido dentro de la función.Además, se eliminaron las columnas innecesarias.

In [35]:
def UsersRecommend(año: int):

    '''
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    '''

    #Se utiliza el dataframe df_recomendaciones_positivas que ya se encuentra filtrado por recommend y sentimiento en el archivo 04_tratamiento_para_funciones.ipynb

    df_filtrado = df_recomendaciones_positivas[df_recomendaciones_positivas['year_review'] == año]

    if df_filtrado.empty:
        return f"No se encontraron juegos para el año ingresado"

    # Agrupo las filas contando las recomendaciones por titulo
    recomendaciones = df_filtrado.groupby('title')['recommend'].count().reset_index()

    # Ordeno en orden descendente
    top_games = recomendaciones.sort_values(by='recommend', ascending=False)

    # Selecciono los 3 juegos principales
    top_3_games = top_games.head(3)

    # resultado
    resultado = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top_3_games['title'])]

    return resultado


In [36]:
UsersRecommend(2015)

[{'Puesto 1: Counter-Strike: Global Offensive'},
 {'Puesto 2: Unturned'},
 {'Puesto 3: Team Fortress 2'}]

 ## **4) Función UsersNotRecommend**

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

En este caso, para obtener el dataset se hizo un merge, se filtraron los recommend=False y los valores de la columna sentimiento igual a 0, por lo que ese filtro no se encuentra incluido dentro de la función. Además, se eliminaron las columnas innecesarias.

In [37]:
def UsersNotRecommend(año):

    '''
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    
    '''
    #Se utiliza el dataframe df_recomendaciones_negativas que ya se encuentra filtrado por recommend y sentimiento en el archivo 04_tratamiento_para_funciones.ipynb
    
    df_filtrado = df_recomendaciones_negativas[df_recomendaciones_negativas['year_review'] == año]

    if df_filtrado.empty:
        return f"No se encontraron juegos para el año ingresado"

    # Agrupo las filas por el nombre del juego y cuento las no recomendaciones
    recomendaciones = df_filtrado.groupby('title')['recommend'].count().reset_index()

    # Ordeno en orden descendente
    bottom_games = recomendaciones.sort_values(by='recommend', ascending=True)

    # Selecciono los 3 juegos principales
    bottom_3_games = bottom_games.head(3)

    # resultado
    resultado = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(bottom_3_games['title'])]

    return resultado

In [38]:
UsersNotRecommend(2015)

[{'Puesto 1: Saints Row 2'},
 {'Puesto 2: Tales of Zestiria'},
 {'Puesto 3: Pillars of Eternity'}]

## **5) Función sentiment_analysis**

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
#Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Para este caso, se hizo el merge de los datos y se exportó eliminando las columnas que no eran necesarias. Dicho proceso se puede ver en el archivo 04_tratamiento_para_funciones.ipynb.

In [39]:
def sentiment_analysis(año):
    '''
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    '''

    #Se utiliza el dataframe df_sentiment_analysis que ya se encuentra filtrado por recommend y sentimiento en el archivo 04_tratamiento_para_funciones.ipynb
    
    # Filtrar el DataFrame por el año de lanzamiento especificado
    df_filtrado = df_sentiment_analysis[df_sentiment_analysis['release_year'] == int(año)]

    if df_filtrado.empty:
        return 'No hay datos para el año ingresado'
    
    # Contar la cantidad de registros de cada categoría de sentimiento
    conteo_sentimiento = df_filtrado['sentimiento'].value_counts().to_dict()

    # Crear un diccionario con etiquetas descriptivas
    resultado = {
        'Negative': conteo_sentimiento.get(0, 0),
        'Neutral': conteo_sentimiento.get(1, 0),
        'Positive': conteo_sentimiento.get(2, 0)
    }

    return resultado

In [40]:
sentiment_analysis(1998)

{'Negative': 23, 'Neutral': 15, 'Positive': 55}